In [ ]:
1

In [ ]:
import conservatoire

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
import conservatoire

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
res = conservatoire.get_results(base, 1)

In [ ]:
for n, df in res[1]:
    print(n)
    display(df[df.prix != df.prix_input])
    assert df[df.prix != df.prix_input].shape[0] == 0

In [ ]:
from utils import StrasbourgSurveyScenario

In [ ]:
if True:
    df = conservatoire.get_df()
    data, extra = conservatoire.build_data(df)
    determine_qf(data["input_data_frame_by_entity"]["famille"])
    scenario = StrasbourgSurveyScenario(base, data)

    names = [
        n
        for n in base.variables
        if n.startswith("strasbourg_conservatoire")
        and not n.endswith("base_ressources")
        and "nombre" not in n
    ]
    for n in names:
        v = scenario.simulation.calculate(n, base_period)
        df[n] = v

    res = (
        df[["index", "MontantFactureSurEleve", *names]]
        .set_index(["index", "MontantFactureSurEleve"])
        .stack()
        .reset_index()
        .rename(columns={0: "Montant calculé", "level_2": "variable"})
    )

    res["match"] = (res["Montant calculé"] - res["MontantFactureSurEleve"]).abs() < 0.1
    ns = res[res.match].groupby("index").variable.last()
    # res.groupby("index").variable.first().to_excel("crr_correspondances_test2.xlsx")
    row_data = pd.DataFrame({"match": res.groupby("index").match.sum()}).reset_index()
    display(row_data.groupby("match").count())

    rdf = df.merge(row_data, on="index")
    display(rdf[rdf.match == 0].groupby("MontantFactureSurEleve").service.count())
    assert len(rdf[rdf.match == 0]) == 0